In [ ]:
## Reading Files
from google.colab import drive

drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import altair as alt
import sklearn
import plotly.express as px
from pandas import Period
from scipy import stats
import re

In [ ]:
ameco_1623 = pd.read_csv('/content/drive/MyDrive/Capstone Data/ameco_data.csv',low_memory=False)
df = ameco_1623.copy()

In [ ]:
df = df[['X.','Customer.No.','Customer.Name','Customer.Category',
                        'Ship.To.State','Invoice.Date','Strikeforce.Flag','Stock.Status',
                        'Item.No.','Item.Group','Product.Classification',
                        'UoM.Code','Quantity','Inventory.Cost','Price',
                        'DPM.Factor','DPM.Price']]

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1155850 entries, 0 to 1155849
Data columns (total 17 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   X.                      1155850 non-null  object 
 1   Customer.No.            1155850 non-null  object 
 2   Customer.Name           1155835 non-null  object 
 3   Customer.Category       590461 non-null   object 
 4   Ship.To.State           571691 non-null   object 
 5   Invoice.Date            1155850 non-null  object 
 6   Strikeforce.Flag        572301 non-null   object 
 7   Stock.Status            691242 non-null   object 
 8   Item.No.                1155840 non-null  object 
 9   Item.Group              572291 non-null   object 
 10  Product.Classification  572290 non-null   object 
 11  UoM.Code                1155835 non-null  object 
 12  Quantity                1155850 non-null  object 
 13  Inventory.Cost          1155847 non-null  object 
 14  Pr

In [ ]:
numerical_columns = ['Inventory.Cost','Price','Quantity']

In [ ]:
df[numerical_columns] = df[numerical_columns].replace({',': ''}, regex=True).astype(float)

In [ ]:
df = df[df['Price'] > 0]

## Dataset Clean

### Customer Category Remap

#### Make the Customer Number consistent

In [ ]:
df['Customer.Name'] = df['Customer.Name'].str.replace(r'[^\w\s]+', '')

<ipython-input-9-ca9b5d3313d0>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Customer.Name'] = df['Customer.Name'].str.replace(r'[^\w\s]+', '')


In [ ]:
df['Customer.Name'] = df['Customer.Name'].map(lambda x: x.strip() if isinstance(x, str) else x)

In [ ]:
df['Customer.Name']

0                             BEN HUR
1                 CUSTOM CONVEYOR INC
2                 CUSTOM CONVEYOR INC
3                 CUSTOM CONVEYOR INC
18                       KENNEDY TANK
                      ...            
1155845    ADS MANUFACTURING OHIO LLC
1155846    ADS MANUFACTURING OHIO LLC
1155847    ADS MANUFACTURING OHIO LLC
1155848    ADS MANUFACTURING OHIO LLC
1155849    ADS MANUFACTURING OHIO LLC
Name: Customer.Name, Length: 1134769, dtype: object

In [ ]:
def map_customer_no(row):
    if len(row['Customer.No.']) == 6 and 'PRE' in row['X.']:
        return 'C' + row['Customer.No.']
    elif len(row['Customer.No.']) == 5 and 'PRE' in row['X.']:
        return 'C0' + row['Customer.No.']
    elif len(row['Customer.No.']) == 4 and 'PRE' in row['X.']:
        return 'C00' + row['Customer.No.']
    elif len(row['Customer.No.']) == 3 and 'PRE' in row['X.']:
        return 'C000' + row['Customer.No.']
    elif len(row['Customer.No.']) == 2 and 'PRE' in row['X.']:
        return 'C0000' + row['Customer.No.']
    elif len(row['Customer.No.']) == 1 and 'PRE' in row['X.']:
        return 'C00000' + row['Customer.No.']
    else:
        return row['Customer.No.']

In [ ]:
df['Customer.No.'] = df.apply(map_customer_no, axis=1)

In [ ]:
print(f"The number of unique customer numberis now {df['Customer.No.'].nunique()}.")
print(f"The number of unique customer name is {df['Customer.Name'].nunique()}.")

The number of unique customer numberis now 2084.
The number of unique customer name is 2123.


In [ ]:
customer_pair = df[['Customer.No.','Customer.Name']]
customer_pair = customer_pair.groupby('Customer.No.')['Customer.Name'].nunique().reset_index().rename(columns={'Customer.Name':'Count'})

In [ ]:
customer_pair[customer_pair['Count']!=1]

,Customer.No.,Count
13,C000109,2
45,C000449,2
63,C000612,2
78,C000767,2
109,C001072,2
...,...,...
1681,C109882,2
1683,C109887,2
1692,C109902,2
1694,C109905,2


In [ ]:
df[df['Customer.No.']=='C001072'].sample(5)

,X.,Customer.No.,Customer.Name,Customer.Category,Ship.To.State,Invoice.Date,Strikeforce.Flag,Stock.Status,Item.No.,Item.Group,Product.Classification,UoM.Code,Quantity,Inventory.Cost,Price,DPM.Factor,DPM.Price
506047,PRE 506048,C001072,APPLIED INDUSTRIAL TECHNOLOGIE,P1,NaN,1/1/2019,NaN,N,ING 212-TK2,NaN,NaN,EA,1.0,34.29,43.59,NaN,35.2
21061,PRE 21062,C001072,APPLIED INDUSTRIAL TECHNOLOGIE,P1,NaN,1/1/2016,NaN,N,ENE M783055,NaN,NaN,EA,1.0,3.62,6.00,NaN,NaN
21060,PRE 21061,C001072,APPLIED INDUSTRIAL TECHNOLOGIE,P1,NaN,1/1/2016,NaN,N,ENE F866028,NaN,NaN,EA,1.0,3.62,6.00,NaN,NaN
91387,PRE 91388,C001072,APPLIED INDUSTRIAL TECHNOLOGIE,P1,NaN,1/1/2016,NaN,N,ENE RSM300K,NaN,NaN,EA,1.0,43.85,80.08,NaN,88
223107,PRE 223108,C001072,APPLIED INDUSTRIAL TECHNOLOGIE,P1,NaN,1/1/2017,NaN,N,ENE RCH202K,NaN,NaN,EA,1.0,33.42,106.50,NaN,71


#### read customer category

In [ ]:
customer_df = pd.read_excel('/content/drive/MyDrive/Capstone Data/20240312 Customer Price List Assignment.xlsx',usecols=[0,1,2,3,4]).copy()

In [ ]:
customer_df.sample(10)

,#,Customer No.,Customer Name,DPM Exempt,Price List Name
3483,3484,C111090,RITCHIE BROTHERS AUCTIONEERS,N,Good
2586,2587,C109977,CHERNE CONSTRUCTION WHSE #157,N,Good
1750,1751,C108912,HERITAGE INDUSTRIAL CONTR.,N,Good
1958,1959,C109240,STEAMFITTERS LOCAL 449 TECH CT,N,Good
840,841,C103797,TAYLOR CRANE & RIGGING INC,N,Good
1190,1191,C106535,WILLIAMS INDUSTRIAL SERV. LLC,N,Good
1335,1336,C107370,EGENOLF INDUSTRIAL GROUP INC.,N,Good
790,791,C103316,E&K EQUIPMENT INC,N,Good
1065,1066,C105641,ALCAR CONSTRUCTORS,N,Good
1264,1265,C107001,IRWIN INDUSTRIES,N,Good


In [ ]:
customer_df['Customer No.'].nunique()

3595

In [ ]:
customer_df[customer_df['Customer No.'] == 'C001072']

,#,Customer No.,Customer Name,DPM Exempt,Price List Name
128,129,C001072,APPLIED INDUSTRIAL TECHNOLOGIES,N,Good


In [ ]:
customer_df['Customer No.'].unique()

array(['C000002', 'C000014', 'C000021', ..., 'C111271', 'C111272',
       'C180650'], dtype=object)

In [ ]:
no_list_1 = list(customer_df['Customer No.'].unique())
no_list_2 = list(df['Customer.No.'].unique())
print(f'Length of the updated file is {len(no_list_1)}')
print(f'Length of the original file is {len(no_list_2)}')
no_list_2 in no_list_1

Length of the updated file is 3595
Length of the original file is 2084


False

In [ ]:
customer_dict = dict(zip(customer_df['Customer No.'],customer_df['Customer Name']))

In [ ]:
def map_customer_name(row):
  if row['Customer.No.'] in customer_dict.keys():
    return customer_dict.get(row['Customer.No.'])
  else:
    return row['Customer.Name']

In [ ]:
df['customer_name'] = df.apply(map_customer_name, axis=1)

#### Map the customer category

In [ ]:
category_dict = dict(zip(customer_df['Customer No.'],customer_df['Price List Name']))

In [ ]:
def map_category(row):
  if row['Customer.No.'] in category_dict.keys():
    return category_dict.get(row['Customer.No.'])
  else:
    return row['Customer.Category']

In [ ]:
df['customer_category'] = df.apply(map_category, axis=1)

In [ ]:
df['customer_category'].value_counts()

Good            611198
Better          333080
Best            170152
Average Cost     17723
List Price        1399
M1                 464
P4                 290
P1                 268
P3                  90
P2                  66
M4                  29
P5                  10
Name: customer_category, dtype: int64

Check whether the customer belongs to a category consistently

> Unfortunately there are about 1507 has inconsistent customer categories.

In [ ]:
df[(df['Customer.Category'] != df['customer_category'])&(~df['Customer.Category'].isnull())]['Customer.No.'].nunique()

1507

#### Mark AMECO sub companies

In [ ]:
def mark_own_company(row):
  if row['customer_name'] in [
    "F&M MAFCO, INC",
    "CHRISTIANSTED EQUIPMENT, LTD.",
    "F&M MAFCO LLC",
    "EQUIPMENTSHARE.COM INC",
    "AMECO IC",
    "AMECO CANADA IC",
    "AMECO IC - VISTRA",
    "AMECO SERVICES INC (TRANSMOUNTAIN)"]:
    return True
  else:
    return False


In [ ]:
df['Own'] = df.apply(mark_own_company,axis=1)

In [ ]:
df['Own'].value_counts()

False    1102450
True       32319
Name: Own, dtype: int64

### Transform Datetime

In [ ]:
## date time
df['Invoice.Date'] = pd.to_datetime(df['Invoice.Date'])
df['Year'] = df['Invoice.Date'].dt.year
df['Month'] = df['Invoice.Date'].dt.month
df['Quarter'] = df['Invoice.Date'].dt.to_period('Q')

In [ ]:
def map_quarter(row):
  if row['Year'] == 2016:
    return row['Quarter']
  elif row ['Year'] == 2020:
    return Period('2019Q1', freq='Q-DEC')
  else:
    return row['Quarter'] - 4
df['Prev_Quarter'] = df.apply(map_quarter, axis=1)
df['Prev_Quarter'] = df['Prev_Quarter'].apply(lambda x: Period(x, freq='Q-DEC'))

### Flag

In [ ]:
## Flag mapping
def stock_flag(row):
    if row['Stock.Status'] == 'S' or row['Stock.Status'] == 'Stock':
        return 'Y'
    elif row['Stock.Status'] == 'N' or row['Stock.Status'] == 'Non Stock':
        return 'N'
    else:
      if row['Strikeforce.Flag'] == 'Y':
        return 'Y'
      elif row['Strikeforce.Flag'] == 'N':
        return 'N'
      else:
        return 'Y'

df['Stock.Flag'] = df.apply(stock_flag, axis=1)

In [ ]:
df.drop(columns={'Strikeforce.Flag','Stock.Flag'},inplace=True)

### Item group & Product Classification

In [ ]:
## Item group and product classification
def fill_missing_most_frequent(group):
    mode_values = group.mode()
    if not mode_values.empty:
        mode_value = mode_values.iloc[0]
        return group.fillna(mode_value)
    else:
        return group

df['Product.Classification'] = df.groupby('Item.No.')['Product.Classification'].transform(fill_missing_most_frequent)
df['Item.Group'] = df.groupby('Item.No.')['Item.Group'].transform(fill_missing_most_frequent)

### Final Dataset

In [ ]:
df.drop(columns=['X.','Customer.Name'],inplace=True)

In [ ]:
df.to_csv('/content/drive/MyDrive/Capstone Data/updated_customer_dataset.csv',index=False)

In [ ]:
df

,Customer.No.,Customer.Category,Ship.To.State,Invoice.Date,Stock.Status,Item.No.,Item.Group,Product.Classification,UoM.Code,Quantity,...,Price,DPM.Factor,DPM.Price,customer_name,customer_category,Own,Year,Month,Quarter,Prev_Quarter
0,C002309,P3,NaN,2016-01-01,N,KCP 14967,NaN,NaN,EA,6.0,...,47.35,NaN,41.46,BEN HUR CONSTRUCTION CO.,Better,False,2016,1,2016Q1,2016Q1
1,C000263,P3,NaN,2016-01-01,N,KNA 977-2PK,Const Supplies CS/SD,HANBOX,PKG,1.0,...,106.26,NaN,0,CUSTOM CONVEYOR INC,P3,False,2016,1,2016Q1,2016Q1
2,C000263,P3,NaN,2016-01-01,S,WES FA-30,Welding Equip WS/SD,APPFLA,EA,4.0,...,69.95,NaN,0,CUSTOM CONVEYOR INC,P3,False,2016,1,2016Q1,2016Q1
3,C000263,P3,NaN,2016-01-01,N,PRO J07520,Const Supplies CS/SD,HANSOC,EA,3.0,...,15.25,NaN,18.8604,CUSTOM CONVEYOR INC,P3,False,2016,1,2016Q1,2016Q1
18,C001501,P3,NaN,2016-01-01,N,LIN ED010216,Welding Equip WS/SD,FILELM,LBS,150.0,...,2.58,NaN,0,KENNEDY TANK,Good,False,2016,1,2016Q1,2016Q1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1155845,C109148,NaN,OH,2021-06-16,NaN,GEC 713SUCB-M,Const Supplies CS/SD,SAFGLO,PKG12,1.0,...,10.98,0.0,0,ADS MANUFACTURING OHIO LLC,Better,False,2021,6,2021Q2,2020Q2
1155846,C109148,NaN,OH,2021-06-16,NaN,BER QT3-45,Welding Equip WS/SD,MIGPAR,EA,2.0,...,49.32,0.0,0,ADS MANUFACTURING OHIO LLC,Better,False,2021,6,2021Q2,2020Q2
1155847,C109148,NaN,OH,2021-06-16,NaN,LAK CTL412-3X,Const Supplies CS/SD,SAFCOV,EA,25.0,...,3.40,0.0,0,ADS MANUFACTURING OHIO LLC,Better,False,2021,6,2021Q2,2020Q2
1155848,C109148,NaN,OH,2021-06-16,NaN,MIL 216326,Welding Equip WS/SD,APPPRT,EA,10.0,...,2.10,0.0,0,ADS MANUFACTURING OHIO LLC,Better,False,2021,6,2021Q2,2020Q2
